In [4]:
from src.WhitespaceTokenizer import WhitespaceTokenizer
import datasets
from transformers import BertConfig, BertForSequenceClassification

dataset = datasets.load_dataset("michaelginn/latent-trees-agreement")
tokenizer = WhitespaceTokenizer()
tokenizer.learn_vocab([row['text'] for row in dataset['train']])
dataset = dataset.map(tokenizer.tokenize_batch, batched=True, load_from_cache_file=False)
print(dataset)

config = BertConfig(vocab_size=tokenizer.vocab_size, num_labels=2, max_position_embeddings=tokenizer.model_max_length)
model = BertForSequenceClassification(config=config).to('mps')

Map:   0%|          | 0/1540 [00:00<?, ? examples/s]

Map:   0%|          | 0/660 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1540
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 660
    })
})


In [2]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
import numpy as np

batch_size = 1
train_epochs = 100

device='mps'

# toy_data = [dataset['train'][0], dataset['train'][26]]
def collate(batch):
    # Convert lists to arrays here and then to tensors
    input_ids = torch.tensor(np.array([item['input_ids'] for item in batch]), device=device)
    labels = torch.tensor(np.array([item['label'] for item in batch]), device=device)
    attention_mask = torch.tensor(np.array([item['attention_mask'] for item in batch]), device=device)
    return {
        'input_ids': input_ids,
        'labels': labels,
        'attention_mask': attention_mask
    }

toy_data = dataset['train'].select([0, 26])
train_dataloader = DataLoader(toy_data, batch_size=2, collate_fn=collate)
optimizer = AdamW(model.parameters(), lr=1e-5)

for epoch in range(train_epochs):
    for batch in train_dataloader:
        optimizer.zero_grad()
        print(batch['input_ids'])
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        print(f"Epoch: {epoch} | Loss: {loss}")
        loss.backward()
        optimizer.step()

# args = TrainingArguments(
#     output_dir=f"../training-checkpoints",
#     evaluation_strategy="epoch",
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     gradient_accumulation_steps=1,
#     save_strategy="epoch",
#     save_total_limit=3,
#     num_train_epochs=train_epochs,
#     load_best_model_at_end=False,
#     logging_strategy='steps',
#     logging_steps=1,
#     report_to='wandb'
# )
#
# trainer = Trainer(
#     model,
#     args,
#     train_dataset=toy_data,
#     eval_dataset=toy_data,
# )

# trainer.train()


/Users/milesper/miniforge3/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


tensor([[ 1,  3,  4,  ...,  0,  0,  0],
        [ 1, 57, 37,  ...,  0,  0,  0]], device='mps:0')
Epoch: 0 | Loss: 0.6844738721847534
tensor([[ 1,  3,  4,  ...,  0,  0,  0],
        [ 1, 57, 37,  ...,  0,  0,  0]], device='mps:0')
Epoch: 1 | Loss: 0.6032574772834778
tensor([[ 1,  3,  4,  ...,  0,  0,  0],
        [ 1, 57, 37,  ...,  0,  0,  0]], device='mps:0')
Epoch: 2 | Loss: 0.688709020614624
tensor([[ 1,  3,  4,  ...,  0,  0,  0],
        [ 1, 57, 37,  ...,  0,  0,  0]], device='mps:0')
Epoch: 3 | Loss: 0.5118123888969421
tensor([[ 1,  3,  4,  ...,  0,  0,  0],
        [ 1, 57, 37,  ...,  0,  0,  0]], device='mps:0')
Epoch: 4 | Loss: 0.6379901170730591
tensor([[ 1,  3,  4,  ...,  0,  0,  0],
        [ 1, 57, 37,  ...,  0,  0,  0]], device='mps:0')
Epoch: 5 | Loss: 0.5188990831375122
tensor([[ 1,  3,  4,  ...,  0,  0,  0],
        [ 1, 57, 37,  ...,  0,  0,  0]], device='mps:0')
Epoch: 6 | Loss: 0.35225194692611694
tensor([[ 1,  3,  4,  ...,  0,  0,  0],
        [ 1, 57, 37,  ...,  0

KeyboardInterrupt: 

In [7]:
longest = 0

for row in dataset['train']:
    tokens = len(tokenizer.tokenize(row['text']))
    if tokens > longest:
        longest = tokens
longest

48